## 데이터(1): 공공자전거 이용정보
- 서울특별시 공공자전거 이용정보 시간대별 데이터
 - 출처: https://data.seoul.go.kr/dataList/OA-15245/F/1/datasetView.do
 - 2017년 ~ 2021년.zip 데이터 활용
 - 8월 머신러닝 > test1

## 데이터 분석 방법론(CRISP-DM)
1. Business Understanding
2. Data Understanding
3. Data Exploration(EDA: 탐색적 데이터 분석)
4. Data Preparation
5. Modeling
6. Evaluation
7. Deployment

## Business Understanding
- 우리나라를 제외하고 대부분의 자전거 대여 사업은 사기업에서 운영(B2C)
 - B2C: 일반 개개인을 대상으로 수입을 얻는 구조
 - 외국 사기업 운영 사례: citibike, Capital bikeshare
 - 구글에 기업명 검색 시 해당 기업 데이터에 접근 가능
- 어떤 사람들이 언제 이용할까?
 - 지난 몇 년 간 이용 형태의 추세/변화는? (코로나 팬데믹 영향 등)
 - 어떤 요인이 사람들의 자전거 이용에 가장 큰 영향을 미칠까? (날씨, 시간 등)
- 이용량 정보가 중요: label 설정
 - 대여소별, 시간대별 이용량 차이를 활용한 효율적인 관리인원 분배의 목적

## Data Understanding
- 가용 데이터로 Business Understanding 단계에서 도출한 방향성이 달성 가능한지 판단
- 현재 보유한 공공자전거 이용정보는 분석이 용이하지 않은 데이터
 - 규격 통일 필요: 파일 제목, 파일 형식/확장자, 연도별 분류 방식 등
 - 대용량 데이터: 소요 시간과 컴퓨터 사양이 문제될 수 있으므로 분석에 필수적인 데이터만 추출/차원 축소

### Data Overview
- csv, xlsx 파일에 접근하여 데이터 확인
- 필요한 데이터를 추출하고 파일 규격을 통일할 때 필요한 인사이트를 얻기 위한 작업

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
file_path = 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1' # '\' -> '/' 변경
file_name = '서울특별시 공공자전거 이용정보(시간대별)_2017년_3.csv' # 크기가 적당한 임의의 파일 선택

In [4]:
pd.read_csv(file_path + '/' + file_name, encoding='cp949')

# 인코딩: 데이터를 안전하게 전달할 목적으로 데이터 변환(암호화 개념 x)
# csv 파일 utf-8(default) 인코딩 오류(한글 깨짐 현상) 발생할 경우 cp949 적용
# 정보가 저장되는 방식과 정보를 읽어들이는 방식이 달라서 발생

,'대여일자','대여시간','대여소번호','대여소','대여구분코드','성별','연령대코드','이용건수','운동량','탄소량','이동거리(M)','이동시간(분)'
0,'2017-08-01','00','538',' 답십리역 8번출구 앞','정기','F','~10대',1,34.06,0.42,1830,13
1,'2017-08-01','00','1006',' 롯데캐슬 115동앞','정기','F','~10대',1,63.32,0.57,2460,13
2,'2017-08-01','00','1323',' 월곡역 입구 육교 밑','정기','F','~10대',1,161.37,1.89,8150,49
3,'2017-08-01','00','1438',' 홈플러스 신내점 앞','정기','F','~10대',1,29.11,0.24,1050,4
4,'2017-08-01','00','115',' 사루비아 빌딩 앞','정기','F','20대',1,13.90,0.18,780,4
...,...,...,...,...,...,...,...,...,...,...,...,...
447376,'2017-08-31','23','500',' 어린이대공원역 3번출구 앞','단체','F','20대',1,23.74,0.25,1090,16
447377,'2017-08-31','23','247',' 당산역 10번출구 앞','단체','M','20대',2,105.22,0.85,3640,53
447378,'2017-08-31','23','415',' DMC역 9번출구 앞','단체','M','20대',2,235.70,1.72,7440,112
447379,'2017-08-31','23','1332',' 석계역 5번출구 건너편','단체','M','30대',2,64.86,0.42,1820,147


### 파일 규격 통일
- 수치 분석이 필요한 대용량 정형 데이터의 경우 확장자를 csv로 통일
 - csv, xlsx 중 처리 속도가 훨씬 빠른 csv 사용
 - 데이터프레임 형태로 분석 용이

#### glob
- 여러 개의 파일(경로)을 한 번에 핸들링하기 위한 도구
- 경로 내 모든 파일을 list 형태로 관리할 수 있어 대용량 파일 접근에 활용

In [6]:
from glob import glob

file_path = 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1'
glob(file_path + '/*.csv') # 경로 내 모든 csv 파일에 접근

['C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\공공자전거 이용정보(시간대별)_19.12_20.056.csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\공공자전거 이용정보(시간대별)_20.06.csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\공공자전거 이용정보(시간대별)_20.07.csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\공공자전거 이용정보(시간대별)_20.08.csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\공공자전거 이용정보(시간대별)_20.09.csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\공공자전거 이용정보(시간대별)_20.10.csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\공공자전거 이용정보(시간대별)_20.11.csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\공공자전거 이용정보(시간대별)_20.12.csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\공공자전거 이용정보(시간대별)_21.01.csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\서울특별시 공공자전거 시간대별 대여정보_201812_201905(1).csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\서울특별시 공공자전거 시간대별 대여정보_201812_201905(2).csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\서울특별시 공공자전거 시간대별 대여정보_201812_201905(3).csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\서울특별시 공공자전거 시간대별 대여정보_201

In [7]:
# csv, xlsx 파일에 모두 접근(csv -> xlsx 순서)
all_files = glob(file_path + '/*.csv') + glob(file_path + '/*.xlsx')
all_files

['C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\공공자전거 이용정보(시간대별)_19.12_20.056.csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\공공자전거 이용정보(시간대별)_20.06.csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\공공자전거 이용정보(시간대별)_20.07.csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\공공자전거 이용정보(시간대별)_20.08.csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\공공자전거 이용정보(시간대별)_20.09.csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\공공자전거 이용정보(시간대별)_20.10.csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\공공자전거 이용정보(시간대별)_20.11.csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\공공자전거 이용정보(시간대별)_20.12.csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\공공자전거 이용정보(시간대별)_21.01.csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\서울특별시 공공자전거 시간대별 대여정보_201812_201905(1).csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\서울특별시 공공자전거 시간대별 대여정보_201812_201905(2).csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\서울특별시 공공자전거 시간대별 대여정보_201812_201905(3).csv',
 'C:/김시원/2022/대학원/KG/8월 머신러닝/1_공공자전거/test1\\서울특별시 공공자전거 시간대별 대여정보_201

In [8]:
# 파일의 개수
len(all_files)

40

In [9]:
pd.read_csv(all_files[0], encoding='cp949') # 첫 번째 파일에 접근(csv)

# 앞서 확인한 2017년_3.csv 파일과 컬럼명(따옴표 유무), 데이터 내용이 모두 상이

,대여일자,대여시간,대여소번호,대여소명,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,사용시간
0,2019-12-01,0,504,504. 신자초교입구교차로,정기권,NaN,~10대,3,74.84,0.67,2880.0,17
1,2019-12-01,0,1029,1029. 롯데 시네마,정기권,NaN,~10대,1,246.71,2.06,8900.0,45
2,2019-12-01,0,1153,"1153. 발산역 1번, 9번 인근 대여소",정기권,NaN,~10대,1,25.60,0.23,1010.0,12
3,2019-12-01,0,512,512. 뚝섬역 1번 출구 옆,정기권,NaN,~10대,1,78.59,0.73,3150.0,17
4,2019-12-01,0,1528,1528. 삼각산동 주민센터,정기권,NaN,~10대,1,26.15,0.22,930.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
6894861,2020-05-31,23,1610,1610. 화랑대역 2번출구 앞,일일권,M,30대,1,231.20,1.65,7120.0,34
6894862,2020-05-31,23,2318,2318. 삼성중앙역4번출구(문화센터더 리빌),일일권,M,30대,1,175.53,1.37,5910.0,40
6894863,2020-05-31,23,2329,2329. 르네상스호텔사거리 역삼지하보도 2번출구,일일권,M,30대,1,0.00,0.00,0.0,20
6894864,2020-05-31,23,1450,1450. 화랑대역 7번출구,일일권,M,40대,1,0.00,0.00,0.0,5


In [10]:
pd.read_excel(all_files[-1]) # 마지막 파일에 접근(xlsx)

# csv 파일 690만 개, xlsx 파일 46만 개의 데이터
# 그럼에도 xlsx 파일을 읽어들일 때 더 오랜 시간 소요
# 대용량 정형 데이터의 경우 csv 파일로 통일시키는 것이 유리

,대여일자,대여시간,대여소번호,대여소명,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,이동시간
0,2018-11-01,0,1361,홍익중고 입구,정기권,F,~10대,1,44.02,0.40,1710,9
1,2018-11-01,0,1403,중화빌딩 앞 (동부시장),정기권,F,~10대,2,21.73,0.22,930,7
2,2018-11-01,0,729,서부식자재마트 건너편,정기권,F,20대,1,36.59,0.36,1540,10
3,2018-11-01,0,747,목동3단지 상가,정기권,F,20대,1,12.47,0.15,630,4
4,2018-11-01,0,505,자양사거리 광진아크로텔 앞,정기권,F,20대,1,22.05,0.27,1160,7
...,...,...,...,...,...,...,...,...,...,...,...,...
465710,2018-11-30,23,2239,교대역 6번출구,일일권,M,30대,1,425.78,2.08,8960,61
465711,2018-11-30,23,1210,롯데월드타워(잠실역2번출구 쪽),일일권,M,30대,1,32.15,0.23,990,5
465712,2018-11-30,23,1162,공항초등학교건너편,일일권,M,40대,1,40.73,0.28,1210,7
465713,2018-11-30,23,1288,문정중교 사거리,일일권,M,60대,1,18.30,0.18,770,3


#### csv/xlsx 파일 처리 속도 비교
- 각각의 파일이 초당 몇 개의 행을 읽어들이는지 비교
- 앞서 직관적으로 확인한 처리 속도를 코드로 증명하는 과정

In [15]:
import time

t0 = time.time() # 이 시점의 시간을 기록
pd.read_csv(all_files[0], encoding='cp949')
t1 = time.time()
pd.read_excel(all_files[-1])
t2 = time.time()

print(t1 - t0) # csv 파일을 읽어들이는 데 소요되는 시간
print(t2 - t1) # xlsx 파일을 읽어들이는 데 소요되는 시간

12.85849666595459
78.95206809043884


In [16]:
# 초당 읽어들이는 행의 개수

csv_rows = 6894866
excel_rows = 465715

csv_row_per_sec = csv_rows // (t1 - t0)
excel_row_per_sec = excel_rows // (t2 - t1)

print(csv_row_per_sec) # csv
print(excel_row_per_sec) # excel

# xlsx 파일이 있다면 csv 파일로 변환하여 핸들링하는 것이 속도 면에서 유리

536210.0
5898.0


### 컬럼명 통일
- 파일 확장자 변환에 앞서 정보 제공 형태(컬럼명, 데이터)를 통일
- 따옴표 유무, 의미는 같지만 이름이 다른 컬럼 통일

In [6]:
# 컬럼의 순서가 같은지 확인
# 같은 의미가 있는 컬럼이 있으면 이름을 동일하게 설정

all_cols = []

for path in all_files:
    
    # csv 파일의 경우
    if '.csv' in path:
        # 컬럼명만 필요하므로 시간을 절약하기 위해 2행 이후부터 스킵
        cols = list(pd.read_csv(path, encoding='cp949', skiprows=lambda x: x > 2).columns)
    
    # excel 파일의 경우
    else:
        cols = list(pd.read_excel(path, skiprows=lambda x: x > 2).columns)
    
    all_cols = all_cols + cols

set(all_cols) # 중복 제거하여 통일이 필요한 컬럼명의 종류를 확인

# 굉장히 오랜 시간 소요 -> 실제 분석은 데이터를 압축하거나 필요한 부분만 추출하여 수행

{"'대여구분코드'",
 "'대여소'",
 "'대여소번호'",
 "'대여시간'",
 "'대여일자'",
 "'성별'",
 "'연령대코드'",
 "'운동량'",
 "'이동거리(M)'",
 "'이동시간(분)'",
 "'이용건수'",
 "'탄소량'",
 '대여구분코드',
 '대여소명',
 '대여소번호',
 '대여시간',
 '대여일자',
 '사용시간',
 '성별',
 '연령대코드',
 '운동량',
 '이동거리',
 '이동시간',
 '이용건수',
 '탄소량'}

#### 컬럼이 분석에 필요한지 판단(Feature Selection)
- 이용량에 대한 정보가 분석에 중요하게 작용한다는 점을 고려하여 변수 선택
- 분석에 사용할 컬럼: 대여일자, 대여시간, 대여구분코드, 성별, 연령대코드
 - 대여소명, 대여소번호: 분석 지역이 서울특별시로 한정되어 있고 이용량과 무관하므로 불필요
 - 나머지 컬럼 역시 이용량과 무관하므로 제외

In [7]:
# 컬럼명 통일에 사용할 컬럼명 목록
reset_cols = all_cols[:12]
reset_cols

['대여일자',
 '대여시간',
 '대여소번호',
 '대여소명',
 '대여구분코드',
 '성별',
 '연령대코드',
 '이용건수',
 '운동량',
 '탄소량',
 '이동거리',
 '사용시간']

In [8]:
# 분석에 사용할 feature 목록
selected_features = ['대여일자', '대여시간', '대여구분코드', '성별', '연령대코드']

In [9]:
all_df = pd.DataFrame() # 데이터를 저장할 빈 데이터프레임 생성

for path in all_files:
    
    # 각각의 파일을 single 변수에 저장
    if '.csv' in path:
        single = pd.read_csv(path, encoding='cp949')
    else:
        single = pd.read_excel(path)
    
    # 컬럼명 재정의(컬럼명 통일)
    single.columns = reset_cols
    
    # 각각의 파일(selected_features)을 빈 데이터프레임(all_df)에 이어붙여서 저장
    all_df = pd.concat([all_df, single[selected_features]])
    print(all_df.shape)
    
    # 데이터 삭제 후 다시 할당
    del single

(6894866, 5)
(9021414, 5)
(11075414, 5)
(12521408, 5)
(14723496, 5)
(17056262, 5)
(18736090, 5)
(19802991, 5)
(20521848, 5)
(21521848, 5)
(22521848, 5)
(23521848, 5)
(23950374, 5)
(24378900, 5)
(25347951, 5)
(26130509, 5)
(26577890, 5)
(27281343, 5)
(27975936, 5)
(28575132, 5)
(29575132, 5)
(30095782, 5)
(31095782, 5)
(32095782, 5)
(33095782, 5)
(34095782, 5)
(35095782, 5)
(36095782, 5)
(37095782, 5)
(38095782, 5)


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_18608\635566610.py:5: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  single = pd.read_csv(path, encoding='cp949')


(39121045, 5)
(39422884, 5)
(40327705, 5)
(41034793, 5)
(41931680, 5)
(42666140, 5)
(43318027, 5)
(43991639, 5)
(44642314, 5)
(45108029, 5)


In [10]:
all_df.to_csv('test1_modified_1.csv', index=False)